In [29]:
import sklearn
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from time import time
from sklearn import tree, metrics
from sklearn.neural_network import MLPClassifier
from sklearn import neighbors
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

In [34]:
#Loading trainData and removing empty entries
trainData = pd.read_csv('train.csv')
trainData.drop(['ID'], axis = 1, inplace = True)
trainData = trainData.dropna() #this can be changed in second iteration
trainData.drop(['Employer_Code'], axis = 1, inplace = True)
# trainEmployerCodeCounts = trainData['Employer_Code'].value_counts().to_dict()

,Gender,DOB,Lead_Creation_Date,City_Code,City_Category,Employer_Category1,Employer_Category2,Monthly_Income,Customer_Existing_Primary_Bank_Code,Primary_Bank_Type,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Approved
1,Male,07/12/86,04/07/16,C10003,A,C,1.0,3500.0,B002,P,Y,S122,G,0.0,20000.0,2.0,13.25,953.0,10,0
6,Male,28/03/73,02/07/16,C10022,B,B,4.0,7500.0,B003,G,Y,S122,C,0.0,130000.0,5.0,14.85,3082.0,10,0
7,Female,12/11/90,03/07/16,C10003,A,A,4.0,3000.0,B006,P,Y,S133,B,0.0,30000.0,3.0,18.25,1088.0,0,0
8,Male,03/03/77,02/07/16,C10003,A,A,4.0,2500.0,B017,G,Y,S133,B,0.0,66000.0,5.0,20.00,1749.0,7,0
10,Male,24/08/83,09/07/16,C10030,B,B,4.0,2700.0,B003,G,Y,S133,B,460.0,20000.0,5.0,18.00,508.0,7,0
11,Male,11/12/86,08/07/16,C10032,C,C,4.0,2899.4,B006,P,Y,S122,E,255.0,30000.0,5.0,15.50,722.0,0,0
18,Male,08/08/92,01/07/16,C10002,A,B,4.0,2284.3,B004,P,Y,S133,E,0.0,26000.0,2.0,20.00,1323.0,4,0
22,Female,28/06/89,01/07/16,C10001,A,C,4.0,7500.0,B001,P,Y,S133,C,0.0,98000.0,3.0,15.50,3421.0,7,0
23,Male,05/09/86,01/07/16,C10003,A,B,4.0,2100.0,B001,P,Y,S122,B,133.3,20000.0,4.0,20.00,609.0,4,0
25,Male,20/06/82,01/07/16,C10001,A,C,4.0,1800.0,B002,P,Y,S133,C,0.0,40000.0,5.0,20.00,1060.0,2,0


In [35]:
#One Hot encoding categorical values
columns_to_be_converted = ['Gender','City_Code','City_Category','Customer_Existing_Primary_Bank_Code','Employer_Category1','Employer_Category2','Primary_Bank_Type','Contacted','Source','Source_Category','Var1'] # removed 'Employer_Code'
#i = 1
for entry in columns_to_be_converted:
    #prefix = 'F'+str(i)
    #trainData[entry] = trainData[entry].cat.add_categories("missing").fillna("missing")
    one_hot = pd.get_dummies(trainData[entry],prefix =entry)
    trainData = trainData.drop(entry, axis=1)
    trainData = trainData.join(one_hot)
    #i +=1

In [36]:
#change Dates
trainData['DOB'] = (pd.to_datetime(trainData['DOB'])-np.datetime64('2018-01-18')).dt.days
trainData['Lead_Creation_Date'] = (pd.to_datetime(trainData['Lead_Creation_Date'])-np.datetime64('2018-01-18')).dt.days

In [80]:
labels_train = trainData['Approved']
trainNumericalValues = trainData.loc[:, 'DOB':'EMI']
trainNumericalHeaders = list(trainNumericalValues)
trainCategoricalValues = trainData.loc[:, 'Gender_Female':]
scaler = MinMaxScaler()
trainNumericalValues = pd.DataFrame(scaler.fit_transform(trainNumericalValues),columns=trainNumericalHeaders)
features_train = pd.concat([trainNumericalValues.reset_index(drop=True),trainCategoricalValues.reset_index(drop=True)],axis=1)
#end of training data preprocessing

In [81]:
testData = pd.read_csv('test.csv')
testData = testData.dropna() #this can be changed in second iteration
testID = testData['ID']
testData.drop(['ID'], axis = 1, inplace = True)
testData.drop(['Employer_Code'], axis = 1, inplace = True)

In [82]:
columns_to_be_converted = ['Gender','City_Code','City_Category','Customer_Existing_Primary_Bank_Code','Employer_Category1','Employer_Category2','Primary_Bank_Type','Contacted','Source','Source_Category','Var1'] # removed 'Employer_Code'
#i = 1
for entry in columns_to_be_converted:
    #prefix = 'F'+str(i)
    #trainData[entry] = trainData[entry].cat.add_categories("missing").fillna("missing")
    one_hot = pd.get_dummies(testData[entry],prefix =entry)
    testData = testData.drop(entry, axis=1)
    testData = testData.join(one_hot)
    #i +=1

In [83]:
testData['DOB'] = (pd.to_datetime(testData['DOB'])-np.datetime64('2018-01-18')).dt.days
testData['Lead_Creation_Date'] = (pd.to_datetime(testData['Lead_Creation_Date'])-np.datetime64('2018-01-18')).dt.days

In [85]:
testNumericalValues = testData.loc[:, 'DOB':'EMI']
testCategoricalValues = testData.loc[:, 'Gender_Female':]
testNumericalHeaders = list(testNumericalValues)
trainNumericalValues = pd.DataFrame(scaler.fit_transform(testNumericalValues),columns=testNumericalHeaders)
features_test = pd.concat([testNumericalValues.reset_index(drop=True),testCategoricalValues.reset_index(drop=True)],axis=1)
features_train

,DOB,Lead_Creation_Date,Monthly_Income,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Gender_Female,Gender_Male,...,Source_Category_B,Source_Category_C,Source_Category_E,Source_Category_F,Source_Category_G,Var1_0,Var1_2,Var1_4,Var1_7,Var1_10
0,0.184969,0.270030,0.000070,0.000000,0.061224,0.2,0.050380,0.062137,0,1,...,0,0,0,0,1,0,0,0,0,1
1,0.051975,0.091988,0.000175,0.000000,0.510204,0.8,0.114354,0.220569,0,1,...,0,1,0,0,0,0,0,0,0,1
2,0.229163,0.178042,0.000057,0.000000,0.102041,0.4,0.250300,0.072183,1,0,...,1,0,0,0,0,1,0,0,0,0
3,0.091320,0.091988,0.000044,0.000000,0.248980,0.8,0.320272,0.121372,0,1,...,1,0,0,0,0,0,0,0,1,0
4,0.156118,0.724036,0.000050,0.007345,0.061224,0.8,0.240304,0.029022,0,1,...,1,0,0,0,0,0,0,0,1,0
5,0.188339,0.632047,0.000055,0.004072,0.102041,0.8,0.140344,0.044947,0,1,...,0,0,1,0,0,1,0,0,0,0
6,0.245767,0.000000,0.000039,0.000000,0.085714,0.2,0.320272,0.089671,0,1,...,0,0,1,0,0,0,0,1,0,0
7,0.214614,0.000000,0.000175,0.000000,0.379592,0.4,0.140344,0.245796,1,0,...,0,1,0,0,0,0,0,0,1,0
8,0.183216,0.000000,0.000034,0.002128,0.061224,0.6,0.320272,0.036538,0,1,...,1,0,0,0,0,0,0,1,0,0
9,0.144337,0.000000,0.000026,0.000000,0.142857,0.8,0.320272,0.070100,0,1,...,0,1,0,0,0,0,1,0,0,0


In [87]:
common = []
trainingColumns = list(features_train)
for col in list(features_test):
    if col in trainingColumns:
        common.append(col)
featuresTestFinal = features_test[common]
features_train_Final = features_train[common]
features_train_Final

,DOB,Lead_Creation_Date,Monthly_Income,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Gender_Female,Gender_Male,...,Source_Category_B,Source_Category_C,Source_Category_E,Source_Category_F,Source_Category_G,Var1_0,Var1_2,Var1_4,Var1_7,Var1_10
0,0.184969,0.270030,0.000070,0.000000,0.061224,0.2,0.050380,0.062137,0,1,...,0,0,0,0,1,0,0,0,0,1
1,0.051975,0.091988,0.000175,0.000000,0.510204,0.8,0.114354,0.220569,0,1,...,0,1,0,0,0,0,0,0,0,1
2,0.229163,0.178042,0.000057,0.000000,0.102041,0.4,0.250300,0.072183,1,0,...,1,0,0,0,0,1,0,0,0,0
3,0.091320,0.091988,0.000044,0.000000,0.248980,0.8,0.320272,0.121372,0,1,...,1,0,0,0,0,0,0,0,1,0
4,0.156118,0.724036,0.000050,0.007345,0.061224,0.8,0.240304,0.029022,0,1,...,1,0,0,0,0,0,0,0,1,0
5,0.188339,0.632047,0.000055,0.004072,0.102041,0.8,0.140344,0.044947,0,1,...,0,0,1,0,0,1,0,0,0,0
6,0.245767,0.000000,0.000039,0.000000,0.085714,0.2,0.320272,0.089671,0,1,...,0,0,1,0,0,0,0,1,0,0
7,0.214614,0.000000,0.000175,0.000000,0.379592,0.4,0.140344,0.245796,1,0,...,0,1,0,0,0,0,0,0,1,0
8,0.183216,0.000000,0.000034,0.002128,0.061224,0.6,0.320272,0.036538,0,1,...,1,0,0,0,0,0,0,1,0,0
9,0.144337,0.000000,0.000026,0.000000,0.142857,0.8,0.320272,0.070100,0,1,...,0,1,0,0,0,0,1,0,0,0


In [89]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 50,30,20,10,5 ,2), random_state=1,early_stopping=True)
clf.fit(features_train_Final, labels_train)
train_pred = clf.predict(features_train_Final)
print("\nClassification report for classifier %s:\n%s\n"
      % (clf, classification_report(labels_train, y_pred)))


Classification report for classifier MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(100, 50, 30, 20, 10, 5, 2),
       learning_rate='constant', learning_rate_init=0.001, max_iter=200,
       momentum=0.9, nesterovs_momentum=True, power_t=0.5, random_state=1,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False):
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     19833
          1       0.81      0.76      0.78       475

avg / total       0.99      0.99      0.99     20308




In [111]:
test_pred = clf.predict(featuresTestFinal)
approve = pd.DataFrame(test_pred)
submission=pd.concat([testID.reset_index(drop=True),approve],axis=1)
submission.columns = ['ID','Approved']
submission.to_csv('prediction.csv')